# Part 1: Grammar

In [1]:
abstract Grammar = {
    cat
        S;
        VP;
        NP;
        Adj;
        Object;
        Person;
        TimeRel;
    fun
        pers2np: Person -> NP;
        obj2np: Object -> NP;
        
        sentence1: NP -> Adj -> S;
        sentence2: NP -> VP -> S;
        sentence3: TimeRel -> NP -> VP -> S;
        
        
        later: TimeRel;
        fabian: Person;
        minor: Adj;
        person_m: Person;
        pocketmoney: Object;
        gun: Object;
        mit: VP -> NP -> VP;
        von: VP -> NP -> VP;
        kaufen: NP -> VP;
        sparen: NP -> VP;
        in_laden_haben: NP -> VP;
}

Successfully imported Grammar.gf

In [2]:
concrete GrammarGer of Grammar = {
  lincat
    S = Str;
    VP = {left: Str; right: Str};
    NP = Str;
    Adj = Str;
    Object = Str;
    Person = Str;
    TimeRel = Str;
  lin
    pers2np p = p;
    obj2np o = o;

    sentence1 np adj = np ++ "ist" ++ adj;
    sentence2 np vp = np ++ vp.left ++ vp.right;
    sentence3 tr np vp = tr ++ vp.left ++ np ++ vp.right;

    later = "später";
    fabian = "Fabian";
    minor = "minderjährig";
    person_m = "M.";
    pocketmoney = ("dem"|"das") ++ "Taschengeld";
    gun = "die Spielzeugpistole";
    mit vp np = {left=vp.left; right=vp.right ++ "mit" ++ np};
    von vp np = {left=vp.left; right=vp.right ++ "von" ++ np};
    kaufen np = {left="kauft"; right=np};
    sparen np = {left="hat"; right=np ++ "gespart"};
    in_laden_haben np = {left="hat"; right=np ++ "in seinem Laden"};
}


Successfully imported GrammarGer.gf

# Part 2: Logic

In [3]:
theory DL_Star_Predicate : latin:/?DL_Star =
    /T A predicate-logic-no-quantifiers version of the deontic logic DL_Star ❙
    //include ☞ur:?DiagramOperators ❙
    tm : type ❙
    event : type ❙
    after : event ⟶ event ⟶ prop ❘ # 1 nach 2 prec 150 ❙
    simultaneous : event ⟶ event ⟶ prop ❘ # 1 zeitgleich 2 prec 150 ❙
    verlauf_axiom : {X,Y,Z} ⊦ (after X Y) ⟶ ⊦ (after Y Z) ⟶ ⊦ (after X Z)❙
    Agent : event ⟶ tm ⟶ prop❙
    Patient :  event ⟶ tm ⟶ prop ❙

    event_exists: (event ⟶ prop) ⟶ prop ❘ # evE 1 ❘ role EventIntroduction ❙
    event_exists_rel: (event ⟶ event ⟶ prop) ⟶ prop ❘ # evEr 1 ❘ role EventRelIntroduction ❙
    von: event ⟶ tm ⟶ prop ❙
    mit: event ⟶ tm ⟶ prop ❙

    proof_sketch : {A: prop} ⊦ A ❙
❚

Successfully imported DL_Star_Predicate.mmt

In [4]:
// the discourse domain theory (ddt) introduces terms, symbols and functions we might need ❚
theory ddt : ?DL_Star_Predicate =    
    fabian: tm ❙
    pers_m: tm ❙
    taschengeld: tm ❙
    spielzeugpistole: tm ❙
    minor: tm ⟶ prop ❙
    kaufevent: event ⟶ prop ❙
    sparevent: event ⟶ prop ❙
    ladenevent : event ⟶ prop ❙
    rückforderungsevent : event ⟶ prop ❙
    disapprove : event ⟶ tm ⟶ prop ❙
    eltern : tm ⟶ tm ❙
❚

Successfully imported ddt.mmt

# Part 3: The semantics construction specifies how to map abstract syntax trees to logical expressions

In [5]:
view GrammarSemantics : http://mathhub.info/tmpGLIF/default/Grammar.gf?Grammar -> ?ddt =
    S = prop ❙
    VP = event ⟶ prop ❙
    NP = tm ❙
    Adj = tm ⟶ prop ❙
    Object = tm ❙
    Person = tm ❙
    TimeRel = event ⟶ event ⟶ prop ❙

    pers2np = [x] x ❙
    obj2np = [x] x ❙
    sentence1 = [np, adj] adj np ❙
    sentence2 = [np, vp] evE [e:event] vp e ∧ Agent e np ❙
    sentence3 = [tr,np,vp] evEr [prev:event, e:event] vp e ∧ Agent e np ∧ tr prev e ❙
    later = after ❙
    fabian = fabian ❙
    minor = minor ❙
    person_m = pers_m ❙
    pocketmoney = taschengeld ❙
    gun = spielzeugpistole ❙
    mit = [vp] [np] [e] (vp e) ∧ (mit e np) ❙
    von = [vp] [np] [e] (vp e) ∧ (von e np) ❙
    kaufen = [s] [e] kaufevent e ∧ Patient e s ❙
    sparen = [s] [e] sparevent e ∧ Patient e s ❙
    in_laden_haben = [s] [e] ladenevent e ∧ Patient e s ❙
❚

Successfully imported GrammarSemantics.mmt

In [6]:
-- examples:
parse "Fabian ist minderjährig" | construct
parse "Fabian hat das Taschengeld gespart" | construct

minor fabian

evE [e:event]((sparevent e)∧(Patient e taschengeld))∧(Agent e fabian)

In [7]:
-- we can use it to populate an MMT theory
parse "Fabian ist minderjährig"
    "M. hat die Spielzeugpistole in seinem Laden"
    "Fabian hat das Taschengeld gespart"
    "später kauft Fabian die Spielzeugpistole von M. mit dem Taschengeld" |
    construct |
    populate -meta=ddt -mode=events

theory generated : http://mathhub.info/tmpGLIF/default?ddt  = s0 : ⊦(minor fabian) ❙ e0 : event ❙ s1 : ⊦(([e:event]((ladenevent e)∧(Patient e spielzeugpistole))∧(Agent e pers_m)) e0) ❙ e1 : event ❙ s2 : ⊦(([e:event]((sparevent e)∧(Patient e taschengeld))∧(Agent e fabian)) e1) ❙ e2 : event ❙ s3 : ⊦(([prev:event][e:event](((((kaufevent e)∧(Patient e spielzeugpistole))∧(von e pers_m))∧(mit e taschengeld))∧(Agent e fabian))∧prev nach e) e2 e1) ❙ ❚

# Manual Formalization

In [30]:
theory fall : ?ddt =
    e0 : event ❙
    e1 : event ❙
    e2 : event ❙
    e3 : event ❙
    
    // M. hat eine Spielzeugpistole in seinem Laden ❙
    c0 : ⊦ ladenevent e0 ∧ Agent e0 pers_m ∧ Patient e0 spielzeugpistole ❙
    
    // Fabian hat das Taschengeld gespart ❙
    c1 : ⊦ sparevent e1 ∧ Agent e1 fabian ∧ Patient e1 taschengeld ❙
    time_sequence1 : ⊦ e1 zeitgleich e0 ❙

    // Später kauft Fabian die Spielzeugpistole von M. mit dem Taschengeld gegen den Willen von Fabians Eltern ❙
    c2 : ⊦ kaufevent e2 ∧ Agent e2 fabian ∧ Patient e2 spielzeugpistole ∧ von e2 pers_m ∧ mit e2 taschengeld ∧
            disapprove e2 (eltern fabian) ❙
    time_sequence2 : ⊦ e2 nach e1 ❙

    // Danach fordert Fabian das Taschengeld von M. zurück ❙
    c3 : ⊦ rückforderungsevent e3 ∧ Agent e3 fabian ∧ Patient e3 taschengeld ∧ von e3 pers_m ❙
    time_sequence3 : ⊦ e3 nach e2 ❙
    
❚

Successfully imported fall.mmt

In [31]:
theory paragraph985_lexicon : ?DL_Star_Predicate =
    eigentuemer : event ⟶ tm ⟶ prop ❙
    besitzer : event ⟶ tm ⟶ prop ❙
    sache : event ⟶ tm ⟶ prop ❙
    herausgabeevent : event ⟶ prop ❙
❚

Successfully imported paragraph985_lexicon.mmt

In [32]:
theory paragraph985_bedingung : ?DL_Star_Predicate =
    include ?paragraph985_lexicon ❙
    bedingungsevent : event ❙
    x : tm ❙
    y : tm ❙
    z : tm ❙
    bedingung : ⊦ herausgabeevent bedingungsevent ∧ eigentuemer bedingungsevent x ∧
            besitzer bedingungsevent y ∧ sache bedingungsevent z ❙
❚

Successfully imported paragraph985_bedingung.mmt

In [33]:
theory paragraph985_folgen : ?DL_Star_Predicate =
    include ?paragraph985_bedingung ❙
    e1 : event ❙
    time : ⊦ e1 nach bedingungsevent ❙
    folge : ⊦ herausgabeevent bedingungsevent ∧ eigentuemer bedingungsevent x ∧ besitzer bedingungsevent y ∧ sache bedingungsevent z ⇒
        strongly_permitted (eigentuemer e1 y ∧ besitzer e1 x ∧ sache e1 z)
❚

Successfully imported paragraph985_folgen.mmt

In [34]:
view subsumption : ?paragraph985_bedingung -> ?fall = 
    eigentuemer = Agent ❙
    besitzer = von ❙
    sache = Patient ❙
    herausgabeevent = rückforderungsevent ❙
    bedingungsevent = e3 ❙
    x = fabian ❙
    y = pers_m ❙
    z = taschengeld ❙
    bedingung = proof_sketch ❙
❚

Successfully imported subsumption.mmt